<a href="https://colab.research.google.com/github/myazdani/high-contrast-baby-images/blob/main/generate_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref: https://fleuret.org/dlc/materials/dlc-slides-9-4-optimizing-inputs.pdf

In [15]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import glob

In [2]:
class MultiScaleEdgeEnergy(nn.Module):
    def __init__(self):
        super().__init__()
        k = torch.exp(- torch.tensor([[-2., -1., 0., 1., 2.]])**2 / 2)
        k = (k.t() @ k).view(1, 1, 5, 5)
        self.register_buffer('gaussian_5x5', k / k.sum())
    def forward(self, x):
        u = x.view(-1, 1, x.size(2), x.size(3))
        result = 0.0
        while min(u.size(2), u.size(3)) > 5:
            blurry = F.conv2d(u, self.gaussian_5x5, padding = 2)
            result += (u - blurry).view(u.size(0), -1).pow(2).sum(1)
            u = F.avg_pool2d(u, kernel_size = 2, padding = 1)
        result = result.view(x.size(0), -1).sum(1)
        return result

In [61]:
#model = models.vgg16(weights = 'IMAGENET1K_V1')
class IntermediateOutputModel(torch.nn.Module):
    def __init__(self, original_model, layer_index=-1):
        super(IntermediateOutputModel, self).__init__()
        self.features = torch.nn.Sequential(*list(original_model.children())[:layer_index])

    def forward(self, x):
        x = self.features(x)
        return x

model = models.vgg16(weights = 'IMAGENET1K_V1')
intermediate_model = IntermediateOutputModel(model.features, layer_index=1)
#model = models.resnet50(weights=None)
intermediate_model.eval()


IntermediateOutputModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

In [69]:
edge_energy = MultiScaleEdgeEnergy()
input = torch.empty(1, 3, 224, 224).normal_(0, 0.01)
input.requires_grad_()
optimizer = torch.optim.Adam([input], lr = 1e-1)


In [70]:
%%time
for k in range(250):
    output = intermediate_model(input)
    score = edge_energy(input) - output[0, 5,0,0]
    #output = model(input)
    #score = edge_energy(input) - output[0, 700] # paper towel
    optimizer.zero_grad()
    score.backward()
    optimizer.step()


CPU times: user 13.2 s, sys: 6.19 ms, total: 13.2 s
Wall time: 13.3 s


In [71]:
output.size(), score.size()

(torch.Size([1, 64, 224, 224]), torch.Size([1]))

In [72]:
result = 0.5 + 0.1 * (input - input.mean()) / input.std()
torchvision.utils.save_image(result, 'dream-course-vgg-untrained.png')